In [ ]:
with open('../inputs/19.txt') as f:
    workflows_data, ratings_data = [_.splitlines() for _ in f.read().split('\n\n')]

In [ ]:
def parse_ratings(ratings):
    return [{k: int(v) for k, v in (rating.split('=') for rating in entry[1:-1].split(','))} for entry in ratings]

In [ ]:
def parse_workflows(workflows):
    flow_map = {}
    
    for workflow in workflows:
        key, flow_str = workflow[:-1].split('{')
        *checks, fallback = flow_str.split(',')
        
        flow = []
        for check in checks:
            condition, effect = check.split(':')
            part, operator, *val_str = condition
            
            flow.append((part, operator, int(''.join(val_str)), effect))
        
        flow_map[key] = (flow, fallback)
        
    return flow_map

In [ ]:
# Part 1
def check_discrete(ratings, flow_key, flows_map, total):
    flow, next = flows_map[flow_key]

    for part, operator, val, effect in flow:
        rating = ratings[part]
        
        passes = (operator == '>' and rating > val) or (operator == '<' and rating < val)
        
        if passes:
            next = effect
            break
    
    if next == 'A':
        return total + sum(ratings.values())
    elif next == 'R':
        return total
    else:
        return total + check_discrete(ratings, next, flows_map, total)
    
def solve_discrete(workflows_data, ratings_data):
    flows_map = parse_workflows(workflows_data)
    ratings_arr = parse_ratings(ratings_data)
    
    total = 0
    for ratings in ratings_arr:
        total += check_discrete(ratings, 'in', flows_map, 0)
        
    return total

solve_discrete(workflows_data, ratings_data)

In [ ]:
# Part 2
def check_range(ranges, key, flows_map, combinations):
    if key == 'R':
        return combinations
    
    if key == 'A':
        c = 1
        for part in 'xmas':
            c *= ranges[part][1] - ranges[part][0] + 1
        return c

    flow, fallback = flows_map[key]
    c = 0
    for part, operator, val, effect in flow:
        lower, upper = ranges[part]

        c += check_range({
            **ranges, 
            part: (
                 val + 1 if operator == '>' else lower,
                 val - 1 if operator == '<' else upper
            )
        }, effect, flows_map, combinations)

        ranges[part] = (
            val if operator == '<' else lower,
            val if operator == '>' else upper    
        )

    return c + check_range(ranges, fallback, flows_map, combinations)
            
    
def solve_range(workflows_data):
    flows_map = parse_workflows(workflows_data)
    
    initial = {
        'x': (1, 4000),
        'm': (1, 4000),
        'a': (1, 4000),
        's': (1, 4000)
    }
            
    return check_range(initial, 'in', flows_map, 0)

solve_range(workflows_data)